## Functions

__Function for getting the dynamic parameters of the cell model__

In [140]:
def get_dyn_param(data,cell_temp):    
    
    ind = np.where(np.array(data['temps']) == cell_temp)[0][0]

    eta = data['etaParam'][ind]
    Q = data['QParam'][ind]
    gamma = data['GParam'][ind]
    M0 = data['M0Param'][ind]
    M = data['MParam'][ind]
    R0 = data['R0Param'][ind]
    RC = data['RCParam'][ind]
    RC = np.exp(-np.divide(1,RC))
    R = data['RParam'][ind]

    return (eta,Q,gamma,M0,M,R0,RC,R)

__SOC calculation for any OCV value Function__

In [141]:
from scipy.interpolate import interp1d

def interp_ocv(ocv,SOC):
    
    dum = interp1d(ocv,SOC)
    
    return dum

In [142]:
def SOC_from_OCV(ocv,temp,OCV,SOC_0,SOC_rel):

    # OCV calculation from the given SOC (Coloumb Counting)

    SOC_0_ocv = interp_ocv(OCV,SOC_0)(ocv)
    SOC_rel_ocv = interp_ocv(OCV,SOC_rel)(ocv)

    soc_ocv = SOC_0_ocv + temp*SOC_rel_ocv
    

    return (soc_ocv)

## Main Code

__Model Data__

In [143]:
model = pd.read_pickle('E2model.pkl')

__Measurement Data__

In [144]:
measurement = pd.read_pickle('E2_DYN_15_P05.pkl')

time = [measurement['script1']['time'] - measurement['script1']['time'][0][0]][0][0]
dt = time[1] - time[0]
current = measurement['script1']['current'][0]
voltage = measurement['script1']['voltage'][0]
soc = measurement['script1']['soc'][0] 

__Initial Values for EKF__

In [145]:
ekf = {}

# Temperature
temp = 5 

# Co-variance values

sigmax = np.diag([1e-6,1e-8,2e-4])
sigmav = 2e-1
sigmaw = 2e-1 

# Cell parameters
ir0 = 0
h = 0
soc = SOC_from_OCV(voltage[0],temp,model['OCV'],model['SOC0'],model['SOCrel']) 

# Matrix storage index
ir_idx = 0
hk_idx = 1
z_idx = 2

Qbump = 5
prior_I = 0
signik = 0

In [146]:
sigmax

array([[1.e-06, 0.e+00, 0.e+00],
       [0.e+00, 1.e-08, 0.e+00],
       [0.e+00, 0.e+00, 2.e-04]])

In [147]:
# EKF Model

ekf['xhat'] = [ir0,h,soc]
ekf['sigmax'] = sigmax
ekf['sigmav'] = sigmav
ekf['sigmaw'] = sigmaw
ekf['Qbump'] = Qbump
ekf['prior_I'] = prior_I
ekf['signik'] = signik

In [179]:
i = 5

vk = voltage[i]
ik = current[i]
Tk = temp


# Cell Parameters calculated from dynamic model

eta = get_dyn_param(model,temp)[0]  
Q = get_dyn_param(model,temp)[1]  
gamma = get_dyn_param(model,temp)[2]  
M0 = get_dyn_param(model,temp)[3]  
M = get_dyn_param(model,temp)[4]  
R0 = get_dyn_param(model,temp)[5]  
RC = get_dyn_param(model,temp)[6]  
R = get_dyn_param(model,temp)[7]

# If cell is charging
if ik<0:
    
    ik = eta*ik
    
    
# Finding the previous model state

xhat = ekf['xhat']
sigmax = ekf['sigmax']
sigmav = ekf['sigmav']
sigmaw = ekf['sigmaw']
Qbump = ekf['Qbump']
I = ekf['prior_I']

if (np.abs(ik) > Q/100):
    
    ekf['signik'] = np.sign(ik)
    
signik = ekf['signik']

# ============= EKF matrix calculation ============= 

nx = len(xhat) + len(RC) -1
Ahat = np.zeros((nx,nx))
Bhat = np.zeros((nx,2))

# SOC EKF matrix
Ahat[len(RC)+z_idx - 1,len(RC)+z_idx -1] = 1
Bhat[z_idx + len(RC) -1 ,0] = -dt/(Q*3600)

# Diffusion current matrix
Ahat[:len(RC),:len(RC)] = np.diag(RC.flatten())
Bhat[:len(RC),0] = (1-RC).T

# Hysterisis matrix
Ah = np.exp(-np.abs(I*gamma*dt/(3600*Q)))
Ahat[len(RC)+hk_idx - 1,len(RC)+hk_idx -1] = Ah


Bhat[len(RC)+hk_idx - 1,0] = (-np.abs(gamma*dt/(Q*3600))*Ah*\
                              (1+np.sign(I)*xhat[len(RC)+hk_idx - 1]))

Bhat[len(RC)+hk_idx - 1,1] = Ah-1








In [180]:
Ahat

array([[0.5776146, 0.       , 0.       ],
       [0.       , 1.       , 0.       ],
       [0.       , 0.       , 1.       ]])

In [181]:
Bhat

array([[ 4.22385401e-01,  0.00000000e+00],
       [-5.72435135e-03,  0.00000000e+00],
       [-5.38799017e-05,  0.00000000e+00]])

In [182]:
Ah

1.0